In [1]:
# Cell loads the data

from dataset_loader import data_loader, get_descriptors, one_filter, data_scaler
import os, sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
from sklearn import preprocessing
# file name and data path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_path = os.getcwd()
file_name = 'data/CrystGrowthDesign_SI.csv'

"""
Data description.

    Descriptors:
        'void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest'
    Source task:
        'H2@100 bar/243K (wt%)'
    Target tasks:
        'H2@100 bar/130K (wt%)' 'CH4@100 bar/298 K (mg/g)' '5 bar Xe mol/kg' '5 bar Kr mol/kg'
"""

descriptor_columns = ['void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest']
one_filter_columns = ['H2@100 bar/243K (wt%)'] 
another_filter_columns = ['H2@100 bar/130K (wt%)'] 

# load data
data = data_loader(base_path, file_name)

# extract descriptors and gas adsorptions
one_property = one_filter(data, one_filter_columns)
descriptors = get_descriptors(data, descriptor_columns)

# prepare training inputs and outputs
X = np.array(descriptors.values, dtype=np.float32)
y = np.array(one_property.values, dtype=np.float32).reshape(len(X), )
X = data_scaler(X)
y = data_scaler(y.reshape(-1, 1)).reshape(len(X),)
# removes catagorical varaiables
test=data.drop(["MOF ID","topology","First nodular character","Second nodular character"],axis=1)
#g_comp=5
# all vs just used !!!! uncomment below for just feature used anaylsis
g_comp=5
test=test[['void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest']]
g=preprocessing.StandardScaler().fit_transform(test)
g=pd.DataFrame(g)
g.columns=test.columns
test=g

In [14]:
from Statistics_helper import net_sample_cluster
t_1,t_2,y_1,y_2=net_sample_cluster(data,['void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest'],'H2@100 bar/77K (g/L)',10)
t_1

2


,void fraction,Vol. S.A.,Grav. S.A.,Pore diameter Limiting,Pore diameter Largest,H2@100 bar/77K (g/L),Cluster
7811,0.656545,-0.867337,0.729837,0.338782,-0.106413,43.3,0
12222,0.656545,-0.749861,0.550632,0.319929,1.081499,44.6,0
4552,0.574321,-0.499957,1.459081,0.018277,-0.153304,46.0,0
9776,0.492097,-0.356850,0.529605,-0.151403,0.464098,51.9,0
5319,0.574321,-0.230830,0.448843,0.008850,0.065522,44.5,0
...,...,...,...,...,...,...,...
1282,0.409872,-0.602482,-0.209197,2.874546,1.917726,44.8,9
10895,0.903217,-1.926760,1.274619,2.393788,1.620748,36.0,9
12472,0.656545,-0.961318,0.686827,1.667937,1.870835,41.3,9
10842,0.903217,-1.875497,0.994582,2.299522,1.480075,36.6,9
